TreeSequences cannot be edited directly. We instead have to edit the TableCollection that translates to the sequence, and then convert the changed TableCollection into a new TreeSequence. For this reason, it's important to understand msprime's Tables API. 

We'll start by importing the tables from the TreeSequences generated in previous notebooks using ```dump_tables()```:

In [1]:
# Import packages.
import msprime, pyslim, numpy, IPython.display

# Create a very basic msprime setup with population structure, test.

population_configurations =[
    msprime.PopulationConfiguration(
    sample_size=3, initial_size=100),
    msprime.PopulationConfiguration(
    sample_size=3, initial_size=100)
]

demographic_events = [
    msprime.MassMigration(
    time=5, source=1, destination=0, proportion=1.0)
]

rho = 1e-4
mu  = 1e-3 

# Simulate with msprime.

ts = msprime.simulate(
    population_configurations=population_configurations,
    demographic_events=demographic_events,
    length=10,
    recombination_rate=rho,
    mutation_rate=mu,
    random_seed=1
)

tabs = ts.dump_tables()

The tables can be printed all at once...

In [3]:
print(tabs)

############################################################
#   Individuals                                            #
############################################################
id	flags	location	metadata
############################################################
#   Nodes                                                  #
############################################################
id	flags	population	individual	time	metadata
0	1	0	-1	0.00000000000000	
1	1	0	-1	0.00000000000000	
2	1	0	-1	0.00000000000000	
3	1	1	-1	0.00000000000000	
4	1	1	-1	0.00000000000000	
5	1	1	-1	0.00000000000000	
6	0	0	-1	65.92482819029316	
7	0	0	-1	71.57784790772325	
8	0	0	-1	87.05704331323707	
9	0	0	-1	125.58636700016072	
10	0	0	-1	265.89952008528115	
11	0	0	-1	487.91558379532819	
############################################################
#   Edges                                                  #
############################################################
id	left		right		parent	child
0	0.00000000	8.

Each table has different properties that I'll go over in turn.

# Table types

## Nodes

In [15]:
print(tabs.nodes)

id	flags	population	individual	time	metadata
0	1	0	-1	0.00000000000000	
1	1	0	-1	0.00000000000000	
2	1	0	-1	0.00000000000000	
3	1	1	-1	0.00000000000000	
4	1	1	-1	0.00000000000000	
5	1	1	-1	0.00000000000000	
6	0	0	-1	65.92482819029316	
7	0	0	-1	71.57784790772325	
8	0	0	-1	87.05704331323707	
9	0	0	-1	125.58636700016072	
10	0	0	-1	265.89952008528115	
11	0	0	-1	487.91558379532819	


```id```: The integer ID of the node.

```flags```: Currently set up to record whether or not the node is a sample: ```IS_SAMPLE = 1```.

```population```: The ID of the population where the individual is born.

**NOTE:** apparently the population ID is supposed to default to -1 if unspecified, but in this table it appears to be defaulting to 0 (which is confusing, as this is also a valid population ID).

```individual```: The ID of the individual that the node belongs to. If unspecified, as here, defaults to -1.

```time```: The birth time of the individual (in backwards-time).

```metadata```: Other binary info specified by the user.

```flags``` and ```time``` are the only two columns that must be non-null. 

## Edges

In [16]:
print(tabs.edges)

id	left		right		parent	child
0	0.00000000	8.80535355	6	4
1	0.00000000	8.80535355	6	5
2	8.80535355	10.00000000	7	1
3	8.80535355	10.00000000	7	5
4	0.00000000	10.00000000	8	0
5	0.00000000	8.80535355	8	1
6	8.80535355	10.00000000	8	7
7	0.00000000	10.00000000	9	3
8	8.80535355	10.00000000	9	4
9	0.00000000	8.80535355	9	6
10	0.00000000	10.00000000	10	8
11	0.00000000	10.00000000	10	9
12	0.00000000	10.00000000	11	2
13	0.00000000	10.00000000	11	10


```id```: The integer ID of the edge.

```left```: The left coordinate of the edge.

```right```: The right coordinate of the edge.

```parent```: The ID of the parent (upper) node on the edge.

```child```: The ID of the child (lower) node on the edge.

## Sites

In [17]:
print(tabs.sites)

id	position	ancestral_state	metadata
0	0.18288280	0	
1	0.59243197	0	
2	0.81714394	0	
3	2.22124548	0	
4	3.15515634	0	
5	4.46134501	0	
6	4.56251638	0	
7	5.93065517	0	
8	6.86500925	0	
9	7.50144312	0	
10	7.61680909	0	
11	7.78389235	0	
12	8.34625673	0	
13	9.88861087	0	


```id```: The integer ID of each site.

```position```: The floating point value specifying the location of each site.

```ancestral_state```: The allelic state at the root of the tree at each site.

```metadata```: Other binary encoded by the user.

## Mutations

In [18]:
print(tabs.mutations)

id	site	node	derived_state	parent	metadata
0	0	2	1	-1	
1	1	8	1	-1	
2	2	1	1	-1	
3	3	10	1	-1	
4	4	2	1	-1	
5	5	10	1	-1	
6	6	1	1	-1	
7	7	9	1	-1	
8	8	2	1	-1	
9	9	2	1	-1	
10	10	1	1	-1	
11	11	0	1	-1	
12	12	2	1	-1	
13	13	2	1	-1	


```site```: The integer ID of the site at which the mutation occurs.

```node```: The ID of the node at which the mutation occurs.

```derived_state```: The new allelic state that arises from the mutation. This is the allele that this node and all descendant nodes will carry, unless there is another subsequent mutation.

```parent```: If applicable, the ID of the mutation that was replaced by this mutation. (Not applicable in the current 'infinite-sites' version of msprime, in which there are infinite sites and mutations do not occur at the same position twice.)

```metadata```: Other binary information encoded by the user. 

## Population table

In [20]:
print(tabs.populations)

id	metadata
0	
1	


## Provenance table

In [22]:
print(tabs.provenances)

id	timestamp	record
0	2018-10-04T10:49:47.044829	{"schema_version": "1.0.0", "software": {"name": "msprime", "version": "0.6.1"}, "parameters": {"command": "simulate", "random_seed": 1, "TODO": "add other simulation parameters"}, "environment": {"libraries": {"gsl": {"version": "2.2"}, "kastore": {"version": "0.1.0"}}, "os": {"system": "Darwin", "node": "6200D-132482-M.local", "release": "17.7.0", "version": "Darwin Kernel Version 17.7.0: Thu Jun 21 22:53:14 PDT 2018; root:xnu-4570.71.2~1/RELEASE_X86_64", "machine": "x86_64"}, "python": {"implementation": "CPython", "version": "3.6.6"}}}


## Individuals

In [23]:
print(tabs.individuals)

id	flags	location	metadata


```id```: The integer ID of each individual.

```flags```: Currently unused, but some of the bits can be used for user-specified stuff.

```location```: Stores the location of an individual in arbitrary dimensions. Can in principle be used to store any continuous-valued quantity, such as phenotype.

```metadata```: Provides a location to store info about each individual.